In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from joblib import Parallel, delayed

#import lightgbm as lgbm
import optuna.integration.lightgbm as lgbm
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold

pd.set_option('max_columns', 150)

In [8]:
def rmspe(preds, ys): return (((preds - ys) / ys) ** 2).mean() ** 0.5

In [3]:
df = pd.read_csv('train.csv')
df

,stock_id,time_id,target,row_id,wap1_mean,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_450,log_return1_realized_volatility_450,log_return2_realized_volatility_450,wap_balance_mean_450,price_spread_mean_450,bid_spread_mean_450,ask_spread_mean_450,total_volume_mean_450,volume_imbalance_mean_450,wap1_mean_300,log_return1_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,wap1_mean_150,log_return1_realized_volatility_150,log_return2_realized_volatility_150,wap_balance_mean_150,price_spread_mean_150,bid_spread_mean_150,ask_spread_mean_150,total_volume_mean_150,volume_imbalance_mean_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0,5,0.004136,0-5,1.003725,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,323.496689,134.894040,1.003482,0.001721,0.004114,0.000366,0.000783,0.000262,-0.000166,263.941176,141.470588,1.003753,0.002953,0.004863,0.000372,0.000822,0.000223,-0.000162,294.928058,137.158273,1.003832,0.003796,0.006087,0.000397,0.000858,0.000188,-0.000147,327.431034,123.586207,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0,11,0.001445,0-11,1.000239,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,411.450000,142.050000,1.000518,0.000918,0.001883,0.000269,0.000348,0.000233,-0.000143,447.981481,97.685185,1.000397,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,484.521739,135.513043,1.000301,0.001058,0.002262,0.000205,0.000353,0.000141,-0.000127,419.277457,151.566474,0.000901,30.0,1289.0,1.900000,0.000501,10.0,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0,16,0.002168,0-16,0.999542,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,416.351064,141.414894,0.998237,0.001158,0.002972,0.000365,0.000605,0.000186,-0.000208,459.113636,156.113636,0.998685,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,455.235294,144.147059,0.999126,0.002138,0.004019,0.000373,0.000679,0.000161,-0.000241,428.537815,132.084034,0.001961,25.0,2161.0,2.720000,0.001048,9.0,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0,31,0.002195,0-31,0.998832,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,435.266667,146.216667,0.998079,0.000993,0.001424,0.000358,0.001058,0.000116,-0.000049,540.000000,146.000000,0.998436,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,418.169811,144.698113,0.998464,0.002196,0.003273,0.000362,0.000920,0.000170,-0.000108,424.234568,151.765432,0.001561,15.0,1962.0,3.933333,0.000802,3.0,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0,62,0.001747,0-62,0.999619,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,343.221591,123.846591,0.999518,0.001378,0.000966,0.000364,0.000519,0.000196,-0.000136,391.944444,117.000000,0.999488,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,407.584270,99.449438,0.999618,0.001609,0.002927,0.000242,0.000395,0.000187,-0.000117,371.266667,131.474074,0.000871,22.0,1791.0,4.045455,0.000360,4.0,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
columns = df.keys().tolist()
[columns.pop(columns.index(c)) for c in  ['time_id', 'row_id', 'target']]

y_data = df['target']
x_data = df[columns]
x_data

,stock_id,wap1_mean,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_450,log_return1_realized_volatility_450,log_return2_realized_volatility_450,wap_balance_mean_450,price_spread_mean_450,bid_spread_mean_450,ask_spread_mean_450,total_volume_mean_450,volume_imbalance_mean_450,wap1_mean_300,log_return1_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,wap1_mean_150,log_return1_realized_volatility_150,log_return2_realized_volatility_150,wap_balance_mean_150,price_spread_mean_150,bid_spread_mean_150,ask_spread_mean_150,total_volume_mean_150,volume_imbalance_mean_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0,1.003725,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,323.496689,134.894040,1.003482,0.001721,0.004114,0.000366,0.000783,0.000262,-0.000166,263.941176,141.470588,1.003753,0.002953,0.004863,0.000372,0.000822,0.000223,-0.000162,294.928058,137.158273,1.003832,0.003796,0.006087,0.000397,0.000858,0.000188,-0.000147,327.431034,123.586207,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0,1.000239,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,411.450000,142.050000,1.000518,0.000918,0.001883,0.000269,0.000348,0.000233,-0.000143,447.981481,97.685185,1.000397,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,484.521739,135.513043,1.000301,0.001058,0.002262,0.000205,0.000353,0.000141,-0.000127,419.277457,151.566474,0.000901,30.0,1289.0,1.900000,0.000501,10.0,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0,0.999542,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,416.351064,141.414894,0.998237,0.001158,0.002972,0.000365,0.000605,0.000186,-0.000208,459.113636,156.113636,0.998685,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,455.235294,144.147059,0.999126,0.002138,0.004019,0.000373,0.000679,0.000161,-0.000241,428.537815,132.084034,0.001961,25.0,2161.0,2.720000,0.001048,9.0,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0,0.998832,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,435.266667,146.216667,0.998079,0.000993,0.001424,0.000358,0.001058,0.000116,-0.000049,540.000000,146.000000,0.998436,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,418.169811,144.698113,0.998464,0.002196,0.003273,0.000362,0.000920,0.000170,-0.000108,424.234568,151.765432,0.001561,15.0,1962.0,3.933333,0.000802,3.0,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0,0.999619,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,343.221591,123.846591,0.999518,0.001378,0.000966,0.000364,0.000519,0.000196,-0.000136,391.944444,117.000000,0.999488,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,407.584270,99.449438,0.999618,0.001609,0.002927,0.000242,0.000395,0.000187,-0.000117,371.266667,131.474074,0.000871,22.0,1791.0,4.045455,0.000360,4.0,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,126,0.999582,0.003691,0.005876,0.000361,0.000878,0.000091,-0.000202,406.045161,161.638710,0.999301,0.001701,0.

In [11]:
kfd = StratifiedKFold(5)
for trn_idx, val_idx in kfd.split(df, y=df.stock_id, groups=df.time_id):
    break
    
x_trn = x_data.iloc[trn_idx]
y_trn = y_data.iloc[trn_idx]

x_val = x_data.iloc[val_idx]
y_val = y_data.iloc[val_idx]

In [12]:
def optuna_lgbm(x_train,
                y_train,
                x_valid,
                y_valid,
                params: dict = None,
                verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    trains = lgbm.Dataset(x_train, y_train)
    valids = lgbm.Dataset(x_valid, y_valid)
    model = lgbm.train(params,
                       trains,
                       valid_sets=valids,
                       num_boost_round=10000,
                       verbose_eval=False,
                       early_stopping_rounds=100)
    best_params = model.params
    oof_pred = model.predict(x_valid)
    score = rmspe(oof_pred, y_valid)
    print('rmspe:', score)
    return oof_pred, model, score, best_params

In [23]:
params = {
        'objective': 'mape',
        'metric': 'mape',
        'num_iterations': 10000,
        'early_stopping_round': 500,
        'n_jobs':6
    }

_, model, score, best_params = optuna_lgbm(x_trn, y_trn,
                                           x_val, y_val, params)

[I 2021-09-18 03:22:52,739] A new study created in memory with name: no-name-b5f69f18-c7ca-4562-a0ea-42469e010650




  0%|                                                                                            | 0/7 [00:00<?, ?it/s]



feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Met 'abs(label) < 1', will convert them to '1' in MAPE objective and metric
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13268
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 53
[LightGBM] [Info] Start training from score 0.003036






feature_fraction, val_score: 0.000721:   0%|                                                     | 0/7 [03:26<?, ?it/s]



feature_fraction, val_score: 0.000721:  14%|######2                                     | 1/7 [03:26<20:37, 206.21s/it][I 2021-09-18 03:26:18,954] Trial 0 finished with value: 0.0007211200302020454 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 0.0007211200302020454.




feature_fraction, val_score: 0.000721:  14%|######2                                     | 1/7 [03:26<20:37, 206.21s/it]

[LightGBM] [Warning] Met 'abs(label) < 1', will convert them to '1' in MAPE objective and metric
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13268
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 53
[LightGBM] [Info] Start training from score 0.003036






feature_fraction, val_score: 0.000721:  14%|######2                                     | 1/7 [11:43<20:37, 206.21s/it]



feature_fraction, val_score: 0.000721:  29%|############5                               | 2/7 [11:43<24:27, 293.46s/it][I 2021-09-18 03:34:36,018] Trial 1 finished with value: 0.0007209748606336821 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 1 with value: 0.0007209748606336821.




feature_fraction, val_score: 0.000721:  29%|############5                               | 2/7 [11:43<24:27, 293.46s/it]

...

[LightGBM] [Warning] Met 'abs(label) < 1', will convert them to '1' in MAPE objective and metric
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007126 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13268
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 53
[LightGBM] [Info] Start training from score 0.003036






min_data_in_leaf, val_score: 0.000716:  60%|##########################4                 | 3/5 [52:07<27:13, 816.83s/it]



min_data_in_leaf, val_score: 0.000716:  80%|###################################2        | 4/5 [52:07<12:34, 754.78s/it][I 2021-09-18 14:08:24,167] Trial 66 finished with value: 0.0007171126904731908 and parameters: {'min_child_samples': 25}. Best is trial 65 with value: 0.0007159754394255936.




min_data_in_leaf, val_score: 0.000716:  80%|###################################2        | 4/5 [52:07<12:34, 754.78s/it]

[LightGBM] [Warning] Met 'abs(label) < 1', will convert them to '1' in MAPE objective and metric
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.042127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13268
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 53
[LightGBM] [Info] Start training from score 0.003036






min_data_in_leaf, val_score: 0.000716:  80%|#################################6        | 4/5 [1:01:21<12:34, 754.78s/it]



min_data_in_leaf, val_score: 0.000716: 100%|##########################################| 5/5 [1:01:21<00:00, 736.28s/it]


rmspe: 0.26350891041127483


In [24]:
best_params

{'objective': 'mape',
 'metric': 'mape',
 'n_jobs': 6,
 'feature_pre_filter': False,
 'lambda_l1': 5.76040714186739,
 'lambda_l2': 0.1435657105138137,
 'num_leaves': 12,
 'feature_fraction': 0.484,
 'bagging_fraction': 0.7795709311102954,
 'bagging_freq': 2,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 500}

In [26]:
save_pickle(model, 'cv2635089_col53_win150_300_450.pkl')

In [25]:
# pickle
def save_pickle(dic, save_path):
    import pickle
    with open(save_path, 'wb') as f:
        # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)


def load_pickle(load_path):
    import pickle
    with open(load_path, 'rb') as f:
        # with gzip.open(load_path, 'rb') as f:
        dic = pickle.load(f)
    return dic